In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

## Useful links regarding assignment, repository and data
- data : http://snap.stanford.edu/data/ca-GrQc.html
- assignment : https://docs.google.com/presentation/d/1xWByuPJcIubdVqwNmKueawuxsWL0bpvKHAFQ5lRd2VM/edit#slide=id.geec04a3a8b_0_100

In [2]:
gqrc = pd.read_csv("data.txt", delimiter = "\t")

In [3]:
information = {
"total nodes" : 5242,
"total edges" : 28980, 
"total triangles" : 48260
}
print("total nodes = 5242")
print("total edges = 28980")
print("total triangles = 48260")
print()
print("edge list")
print()
print(gqrc.head())

total nodes = 5242
total edges = 28980
total triangles = 48260

edge list

   FromNodeId  ToNodeId
0        3466       937
1        3466      5233
2        3466      8579
3        3466     10310
4        3466     15931


## Applying Chiba-Nishizeki Algorithm

- Paper : http://www.ecei.tohoku.ac.jp/alg/nishizeki/sub/j/DVD/PDF_J/J053.pdf
- we apply the algorithm to find the number of triangles 

![chiba nishizeki algorithm](cn.png)

#### Picking nodes of same color

In [4]:
def degree_sorted_nodes(dictionary): # sorts the nodes in decreasing order of their degrees
    return {k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1], reverse = True)}


def chiba_nishizeki(data):

    graph_data = data["edgelist"] # columns = [FromNodeId, ToNodeId]
    graph_data = graph_data[graph_data["FromNodeId"]!=graph_data["ToNodeId"]]
    edge_list = graph_data
    nodes = np.unique(graph_data["FromNodeId"])
    node_degree = dict()
    for x in nodes:
        node_degree[x] = graph_data[graph_data["FromNodeId"] == x].shape[0]
    nodes = degree_sorted_nodes(node_degree)
    nodeIDs = list(nodes.keys())
    triangles = []
    for i in range(len(nodeIDs)-2):
        node = nodeIDs[i]
        node_neighbours = np.unique(list(edge_list[edge_list["FromNodeId"] == node]["ToNodeId"]))
        marked_nodes = list(node_neighbours)
        for neighbour in node_neighbours:
            neighbour_neighbours = np.unique(list(edge_list[edge_list["FromNodeId"]==neighbour]["ToNodeId"]))
            for third_neighbour in neighbour_neighbours:
                if(third_neighbour in marked_nodes):
                    triangles.append([node,neighbour,third_neighbour])
            marked_nodes.remove(neighbour)
        edge_list = edge_list[edge_list["FromNodeId"]!=node]
        edge_list = edge_list[edge_list["ToNodeId"]!=node]


    print("_____________________________________________________")
    print("total triangles = ",len(triangles))
    print("_____________________________________________________")


In [5]:
data = {
        "edgelist" : gqrc,
        }

chiba_nishizeki(data)

_____________________________________________________
total triangles =  48260
_____________________________________________________


In [12]:
!python -m cProfile -o leapfrog_grqc.prof chiba_nishizeki.py

In [13]:
!flameprof leapfrog_grqc.prof > leapfrog_grqc.svg

/System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: No module named flameprof


In [11]:
!pip install flameprof

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
